In [13]:
import pickle
#from loadData import dataLoader
import tensorflow as tf
from CalculateEER import SKGetEER as GetEER
import numpy as np

In [4]:
def onehot(x,numlabels):
    t = [0 for i in range(numlabels)]
    t[x-1] = 1
    return t

In [19]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
data= pickle.load(open('/Users/chaitanya/Documents/python/keystrokes/data_augmentation/greyc1/PCA_GREYC_2X.pickle','rb'))


In [22]:
X = data['data']
X = np.array(X).reshape(len(X), 1, 28, 1)
y = data['labels']
#y_train = [onehot(x,100) for x in train_data['labels']]
#X_test = np.array(test_data['features']).reshape(len(test_data['features']), 1, 28, 1)
#y_test = [onehot(x,100) for x in test_data['labels']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=122)
print([len(x) for x in [X_test, X_val, y_test, y_val, X_train, y_train]])

[2222, 2223, 2222, 2223, 10371, 10371]


In [23]:
X[1]


array([[[-4.73084907e+00],
        [ 2.95269356e-01],
        [-6.11415929e-01],
        [-4.28988830e-01],
        [-5.43085855e-01],
        [-4.77305998e-01],
        [ 8.66831428e-01],
        [-4.69077402e-03],
        [ 4.15252713e-01],
        [-5.61836442e-02],
        [ 1.33056774e-02],
        [-7.25857825e-01],
        [ 2.83767511e-01],
        [ 2.50457215e-01],
        [-1.92024859e-01],
        [ 6.48817185e-01],
        [-4.54668180e-01],
        [ 3.48334184e-01],
        [ 2.12340031e-01],
        [ 2.51877350e-01],
        [ 5.16093974e-02],
        [-1.19143691e-01],
        [-3.03732109e-03],
        [ 1.05386577e-01],
        [-1.87127972e-01],
        [-1.99883006e-01],
        [-1.26656182e-01],
        [ 4.07768507e-02]]])

In [24]:
tf.reset_default_graph()
inp = tf.placeholder(tf.float32, shape=[None, 1, 28, 1], name = "inp")
out = tf.placeholder(tf.float32, shape = [None,100], name = "out")


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([1, 5, 1, 20])
b_conv1 = bias_variable([20])

h_conv1 = tf.nn.relu(conv2d(inp, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([1, 5, 20, 128])
b_conv2 = bias_variable([128])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

print(h_pool2.shape)
W_fc1 = weight_variable([1 * 7 * 128, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 1*7*128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 100])
b_fc2 = bias_variable([100])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_ = tf.nn.softmax(y_conv)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=out, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(out, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

(?, 1, 7, 128)


In [56]:
X.shape[0]

20241

In [25]:
with tf.Session() as sess:
    try:
        sess.run(tf.global_variables_initializer())
        batchSize = 64
        for epoch in range(500):
            j = -1
            i = 0
            while i < X_train.shape[0]:
                j += 1
                batch = X_train[i: i + batchSize]
                batch_labels = y_train[i: i + batchSize]
                i = i + batchSize
                #print(h_pool2.eval(feed_dict = {inp: batch, keep_prob: 1.0}).shape)
                sess.run(train_step, feed_dict={inp: batch, out: batch_labels, keep_prob: 0.5})
                if not j%50:
                    #print("here")
                    train_accuracy = accuracy.eval(feed_dict={inp: batch, out: batch_labels, keep_prob: 1.0})
                    train_out = sess.run(y_, feed_dict={inp: batch, out: batch_labels, keep_prob: 1.0})
                    valid_accuracy = accuracy.eval(feed_dict={inp: X_val, out: y_val, keep_prob: 1.0})
                    valid_out = sess.run(y_, feed_dict={inp: X_val, out: y_val, keep_prob: 1.0})
                    #print(valid_out[0])
                    test_accuracy = accuracy.eval(feed_dict={inp: X_test, out: y_test, keep_prob: 1.0})
                    test_out = sess.run(y_, feed_dict={inp: X_test, out: y_test, keep_prob: 1.0})
                    TEER = GetEER(np.array(test_out), np.array(y_test))
                    VEER = GetEER(np.array(valid_out), np.array(y_val))
                    print(epoch, j, train_accuracy, valid_accuracy, test_accuracy, VEER, TEER)
    except Exception as e:
        print(e)
        d = {}
        d['cnn'] = sess.run(y_, feed_dict={inp: data.data, out: data.data_labels, keep_prob: 1.0})
        d['y'] = data.data_labels
        pickle.dump(d, open("./Data/CNNGREYC1.pickle", "wb"))


0 0 0.015625 0.009896536 0.007650765 0.48959227906596325 0.5100691887370555
0 50 0.046875 0.04948268 0.053555354 0.35668789808917195 0.360290776919582
0 100 0.09375 0.15294647 0.15661566 0.2532969531605275 0.25362976406533577
0 150 0.234375 0.24741341 0.2569757 0.18194192377495463 0.18309859154929578
1 0 0.265625 0.26675662 0.27677768 0.1709090909090909 0.17454545454545456
1 50 0.40625 0.34862798 0.36228624 0.14012738853503184 0.1386994088221919
1 100 0.5 0.46018893 0.47164717 0.12136363636363637 0.12176283507496592
1 150 0.640625 0.5272155 0.5360036 0.10437556973564266 0.10163339382940109
2 0 0.59375 0.54475933 0.55625564 0.10113378684807256 0.09754990925589836
2 50 0.546875 0.5780477 0.5931593 0.09215328467153285 0.08626198083067092
2 100 0.765625 0.63922626 0.65031505 0.08219800181653043 0.07891156462585033
2 150 0.703125 0.65497077 0.6728173 0.07727272727272727 0.07301587301587302
3 0 0.765625 0.66891587 0.680018 0.07615593834995467 0.07129881925522252
3 50 0.71875 0.6882591 0.7016

27 50 0.9375 0.8776428 0.89243925 0.02470498961727032 0.023643273418250916
27 100 1.0 0.877193 0.8910891 0.024850393271445902 0.02265681113565902
27 150 0.96875 0.8807917 0.8874887 0.024545954370515773 0.02415696115066052
28 0 0.9375 0.8785425 0.8888389 0.02328276012486539 0.023597814326887234
28 50 0.96875 0.8785425 0.89243925 0.02403704158090123 0.022543163407249815
28 100 1.0 0.87809265 0.89468944 0.024954902147884605 0.021697624307885334
28 150 0.96875 0.87809265 0.890189 0.023918901111883568 0.0233023302330233
29 0 0.921875 0.8785425 0.8888389 0.023073742371987987 0.02298411659347753
29 50 0.9375 0.87989205 0.89243925 0.022869268483303572 0.021270308849066725
29 100 1.0 0.87989205 0.8960396 0.02444598935827006 0.02235223522352235
29 150 0.96875 0.88304096 0.89063907 0.023364549680339155 0.02133849748611225
30 0 0.9375 0.8794422 0.889739 0.022937426446198377 0.020965732936930055
30 50 0.96875 0.88394064 0.8973897 0.022392162743039935 0.02165671112565802
30 100 1.0 0.8834908 0.89783

55 0 0.984375 0.8987854 0.9122412 0.016739595686964107 0.017424469719699243
55 50 1.0 0.90238416 0.91134113 0.017471157822035014 0.016678940621334862
55 100 1.0 0.9046334 0.9122412 0.018002789932614494 0.017147169262380784
55 150 1.0 0.9014845 0.91134113 0.017193982106262808 0.016274354708198092
56 0 0.984375 0.902834 0.90909094 0.017239420748192676 0.016869868805062326
56 50 1.0 0.902834 0.9144915 0.01857077295673787 0.017719953813563175
56 100 1.0 0.900135 0.9126913 0.01757112283428073 0.018447299275382082
56 150 1.0 0.8987854 0.909991 0.017330298032052417 0.017347189264380985
57 0 0.984375 0.89923525 0.90729076 0.018166369043562027 0.01700624607915337
57 50 1.0 0.8974359 0.91134113 0.01755294737750878 0.017569938812063025
57 100 1.0 0.90328383 0.90954095 0.01711673641498203 0.018883706552473428
57 150 1.0 0.9014845 0.9077408 0.01785738627843891 0.016933511532971478
58 0 0.984375 0.90238416 0.9059406 0.017430263044298133 0.017797234268881433
58 50 1.0 0.9005848 0.9108911 0.0167804904

83 100 1.0 0.9127305 0.9158416 0.015712682379349047 0.015824309703697644
83 150 1.0 0.91363025 0.9189919 0.0154491382561558 0.016265262889925355
84 0 1.0 0.908682 0.91629165 0.015685419194191124 0.016051605160516052
84 50 1.0 0.9127305 0.91854185 0.016317016317016316 0.01622889561683441
84 100 1.0 0.91497976 0.91539156 0.01631247245282333 0.01555610106465192
84 150 1.0 0.9100315 0.9158416 0.016319129646418858 0.015715207884424806
85 0 1.0 0.9100315 0.9149415 0.015349173243910086 0.01600614606915237
85 50 1.0 0.9073324 0.91719174 0.014926593873962295 0.015742483339243014
85 100 1.0 0.908682 0.91854185 0.01622159516896359 0.015383356517469929
85 150 1.0 0.90958166 0.91809183 0.01672142023019216 0.015510641973288239
86 0 1.0 0.9091318 0.9144915 0.016339735637981252 0.015842493340243114
86 50 1.0 0.908682 0.9140414 0.01559908577452437 0.014887852421605797
86 100 1.0 0.91363025 0.9158416 0.01672596409438515 0.014833301511969378
86 150 1.0 0.91048133 0.9144915 0.016671437724069302 0.01776541

111 150 1.0 0.9140801 0.91854185 0.01581264739159476 0.015392448335742666
112 0 1.0 0.9131804 0.919892 0.015367348700682034 0.015947049250379583
112 50 1.0 0.9100315 0.9212421 0.015394611885839956 0.01573793743010665
112 100 1.0 0.9154296 0.9216922 0.013876961245382298 0.01627890061733446
112 150 1.0 0.90958166 0.91944194 0.01376790850475061 0.016408386508659983
113 0 1.0 0.9091318 0.91944194 0.015476401441313722 0.015337897426106247
113 50 1.0 0.9091318 0.9216922 0.016321560181209303 0.014865122875923956
113 100 1.0 0.91183084 0.9216922 0.015362804836489047 0.016678940621334862
113 150 1.0 0.91183084 0.91944194 0.015467313712927748 0.015346989244378984
114 0 1.0 0.9127305 0.91944194 0.014958400923313204 0.016374364709198192
114 50 1.0 0.9122807 0.91809183 0.014881155232032425 0.015256071061651619
114 100 1.0 0.9131804 0.919892 0.01493568160234827 0.015060596968787789
114 150 1.0 0.91048133 0.91629165 0.014731207713663853 0.015601560156015602
115 0 1.0 0.90958166 0.91629165 0.015508208

...

168 0 1.0 0.91183084 0.9234924 0.014544909281751388 0.015224249697697043
168 50 1.0 0.91183084 0.9212421 0.014217751059856324 0.017669948813063125
168 100 1.0 0.91183084 0.9239424 0.014117786047610608 0.015437907427106348
168 150 1.0 0.911381 0.9225923 0.015022015022015021 0.016597114256880232
169 0 1.0 0.91363025 0.9207921 0.01546276984873476 0.01678349653147133
169 50 1.0 0.9167791 0.92439246 0.014172312417926453 0.015310621971288038
169 100 1.0 0.9140801 0.9212421 0.013699750541855805 0.01583340152197038
169 150 1.0 0.91767883 0.9216922 0.013863329652803336 0.015969778796061426
170 0 1.0 0.91767883 0.9248425 0.014653962022383076 0.01555352241537054
170 50 1.0 0.91632926 0.9207921 0.014803909540751645 0.015497004245879133
170 100 1.0 0.9154296 0.9221422 0.014613067244646193 0.015392448335742666
170 150 1.0 0.91632926 0.9212421 0.01467668134334801 0.014869668785060325
171 0 1.0 0.9154296 0.9216922 0.013308978221258923 0.015424269699697242
171 50 1.0 0.9122807 0.91944194 0.014576716331

196 0 1.0 0.919928 0.920342 0.013849698060224376 0.014083226504468629
196 50 1.0 0.91902834 0.91944194 0.013654311899925935 0.014319613779559775
196 100 1.0 0.919928 0.9216922 0.01399510171439996 0.015174244697196993
196 150 1.0 0.9194782 0.9216922 0.013522539838329312 0.015210611970287937
197 0 1.0 0.91767883 0.9216922 0.013954206936663077 0.016042513342243315
197 50 1.0 0.917229 0.9221422 0.01358615393703113 0.01515606106065152
197 100 1.0 0.917229 0.9230423 0.013917856023119181 0.015847039249379483
197 150 1.0 0.9167791 0.9216922 0.012863679530346197 0.014419623780559875
198 0 1.0 0.91902834 0.9230423 0.0131772061596623 0.014483266508469029
198 50 1.0 0.91767883 0.92529255 0.01296364454259191 0.014065042867923157
198 100 1.0 0.917229 0.9212421 0.014549453145944373 0.01485603105765122
198 150 1.0 0.91767883 0.92574257 0.013608873257996064 0.01532880560783351
199 0 1.0 0.91902834 0.92619264 0.014608523380453205 0.014701470147014702
199 50 1.0 0.9167791 0.92439246 0.014545454545454545 

224 50 1.0 0.9208277 0.92529255 0.014063259677294765 0.01482875560283301
224 100 1.0 0.9208277 0.9248425 0.01323596531264263 0.015287892425606197
224 150 1.0 0.9185785 0.9234924 0.014331347664680997 0.016015237887425107
225 0 1.0 0.9194782 0.9230423 0.013931487615698143 0.015542463337242815
225 50 1.0 0.9167791 0.9234924 0.014522189960786452 0.015151515151515152
225 100 1.0 0.91632926 0.9212421 0.015290103009401255 0.014815117875423906
225 150 1.0 0.92037785 0.9239424 0.015419501133786848 0.01515606106065152
226 0 1.0 0.9208277 0.9239424 0.01522194504650645 0.014974224695196792
226 50 1.0 0.9208277 0.92574257 0.014249558109207233 0.01400594604915037
226 100 1.0 0.92037785 0.9248425 0.013808803282487493 0.014210511960286937
226 150 1.0 0.91767883 0.9230423 0.014853892046874503 0.014501450145014501
227 0 1.0 0.91902834 0.9230423 0.01472211998527788 0.014983316513469529
227 50 1.0 0.91767883 0.9189919 0.014744839306242815 0.01548336651847003
227 100 1.0 0.92037785 0.9212421 0.015912612403

252 50 1.0 0.9185785 0.9239424 0.01331352208545191 0.014410531962287138
252 100 1.0 0.92037785 0.9230423 0.014285909022751129 0.015087872423605996
252 150 1.0 0.9212776 0.9225923 0.013795171689908532 0.015274254698197093
253 0 1.0 0.92217726 0.9212421 0.014113242183417621 0.01527880060733346
253 50 1.0 0.91767883 0.9225923 0.014022364899557882 0.016187982434607097
253 100 1.0 0.917229 0.9239424 0.01381334714668048 0.015333351516969879
253 150 1.0 0.91767883 0.92439246 0.014422224948540738 0.01602887561483421
254 0 1.0 0.91767883 0.9239424 0.014113242183417621 0.01596068697778869
254 50 1.0 0.92037785 0.9239424 0.011982169876906718 0.015592468337742865
254 100 1.0 0.917229 0.920342 0.014017821035364895 0.014792388329742065
254 150 1.0 0.91767883 0.9225923 0.014817541133330607 0.014246879233377883
255 0 1.0 0.91812867 0.9216922 0.015371892564875022 0.014560546963787288
255 50 1.0 0.91812867 0.9189919 0.013740645319592688 0.016351635163516353
255 100 1.0 0.9154296 0.9207921 0.014135961504

KeyboardInterrupt: 